### total 12,447개

미국 - 1042개(뉴스)  
미국 - 1094개(블로그)  

여행 - 991개(뉴스)  
여행 - 1098개(블로그)  

네이버 - 815개(뉴스)  
네이버 - 1944개(블로그)  

네이버블로그 - 453개(뉴스)  
네이버블로그 - 975개(블로그)  

네이버쇼핑 - 300개(뉴스)  
네이버쇼핑 - 1000개(블로그)  

네이버지도 - 508개(뉴스)  
네이버지도 - 924개(블로그)  

네이버클라우드 - 376개(뉴스)  
네이버클라우드 - 927개(블로그)  

***import***

In [1]:
import numpy as np
import pandas as pd
import pykospacing
spacer = pykospacing.Spacing()
from konlpy.tag import Okt
okt = Okt()
import re
from gensim import corpora
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

**데이터 불러오기**

In [2]:
NAVERMAP_news = pd.read_csv('네이버지도(뉴스)-508row.csv')
NAVERMAP_blog = pd.read_csv('네이버지도(블로그)-924row.csv')

In [3]:
NAVERMAP_news

,title,link,description
0,광고로 물든 '녹색창'… 점유율 하락중인 네이버[양철민의 아알못],https://n.news.naver.com/mnews/article/011/000...,\n\n네이버에 '우크라이나' 키워드 검색시맥락없는 '결혼광고'가 최상단에 노출반년...
1,"구글 진격에 카톡·네이버 '흔들'…유튜브, 첫 사용자 1위 눈앞(종합)",https://n.news.naver.com/mnews/article/001/001...,"\n\n카톡, 유튜브와 5월 모바일 MAU 격차 역대 최소…추세대로면 하반기에 추월..."
2,"유튜브, 첫 사용자 1위 눈앞…진격하는 구글에 카톡·네이버 `흔들`",https://n.news.naver.com/mnews/article/029/000...,"\n\n카톡, 유튜브와 5월 모바일 MAU 격차 역대 최소이 추세 대로라면 하반기에..."
3,서울시 경보문자보다 네이버 장애가 더 떨렸다[사이다IT],https://n.news.naver.com/mnews/article/003/001...,\n\n'뜬금' 없는 재난문자보다 네이버 앱 장애에 공포감플랫폼 의존사회 심화…포털...
4,"대학생 여행 정보 탐색, 네이버보다 '인스타·유튜브'서 더 한다",https://n.news.naver.com/mnews/article/092/000...,\n\n비누랩스 에브리타임 1천명 조사...15% 돈·시간 없어 여행 계획 안 세워...
...,...,...,...
503,'육성하고 협업까지'...네이버식 자율주행 시장 공략법,https://n.news.naver.com/mnews/article/293/000...,\n\n\n\n\n\n네이버랩스 디지털 트윈 기술로 구축한 실제 서울 상암지역의 정...
504,"네이버클라우드-네이버랩스-모라이, 자율주행 시뮬레이션 위해 뭉쳤다",https://n.news.naver.com/mnews/article/016/000...,"\n\n\n\n\n\n네이버클라우드가 네이버랩스, 자율주행 시뮬레이션 업체 모라이(..."
505,"네이버클라우드·네이버랩스, 모라이와 맞손…자율주행 시뮬레이션 활용 강화",https://n.news.naver.com/mnews/article/011/000...,\n\n모라이의 자율주행 시뮬레이션 솔루션에네이버랩스 정밀지도 제작 솔루션 접목\n...
506,지도 길찾기에 예매 기능 탑재...코레일 `고객 체감` 서비스 실시,https://n.news.naver.com/mnews/article/029/000...,"\n\n기차 연착 시간, 코레일톡으로 확인기차 이용 승객의 여름휴가 짐, 숙소 배송..."


### 전처리 함수 
*6주차 01.WordCloud 실습코드 참고*

- <span style = 'font-size:1.1em;line-height:1.5em'>(1) 줄바꿈 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(2) 오탈자 교정 (생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(3) 띄어쓰기 교정</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(4) sentence tokenization (너무 오래걸려서 생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(5) 특수문자 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(6) word tokenization (형태소 분석 및 stemming)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(7) stopword removal</span>

**불용어 목록 생성**

In [4]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거

In [5]:
def preprocessing(x):
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    results = []
    result = []
    x = x.replace('\n',' ') # (1) 줄바꿈 문자 제거
    df_news = spacer(x) # (3) 띄어쓰기 교정
    
    tokens = okt.morphs(x, stem=True) # (6) word tokenization
    for token in tokens:
        token = p.sub('', token) # (5) 특수문자 제거
        if token not in stopwords:
            result.append(token) # (7) stopword removal
    results.extend(result)
    results = ' '.join(results)
    return results

### pos_taging 함수

#(1) 품사 태깅 수행 

#(2) 명사, 형용사, 부사에 해당하는 단어들만 추출하여 저장

In [6]:
def pos_taging(x):
    pos_news = okt.pos(x) # (1)
    filtered_result = [word for word, pos in pos_news if pos in 'Noun'] # (2)
    nouns_news = [n for n in filtered_result if len(n) > 1]
    return nouns_news

### 전처리 및 postag

In [7]:
NAVERMAP_news['preprocessed_description'] = NAVERMAP_news['description'].apply(lambda x: preprocessing(x))
NAVERMAP_news['preprocessed_description'] = NAVERMAP_news['preprocessed_description'].apply(lambda x: pos_taging(x))

In [8]:
NAVERMAP_blog['preprocessed_description'] = NAVERMAP_blog['description'].apply(lambda x: preprocessing(x))
NAVERMAP_blog['preprocessed_description'] = NAVERMAP_blog['preprocessed_description'].apply(lambda x: pos_taging(x))

In [9]:
NAVERMAP_news['preprocessed_description']

0      [네이버, 우크라이나, 워드, 검색, 맥락, 결혼, 광고, 최상, 노출, 반년, 사...
1      [카톡, 유튜브, 모바일, 격차, 역대, 최소, 추세, 대로, 하반기, 추월, 사용...
2      [카톡, 유튜브, 모바일, 격차, 역대, 최소, 추세, 대로, 라면, 하반기, 추월...
3      [뜬금, 재난, 문자, 네이버, 장애, 공포, 플랫폼, 의존, 사회, 심화, 포털,...
4      [비누, 랩스, 에브리, 타임, 조사, 여행, 계획, 세대, 장소, 여행, 계획, ...
                             ...                        
503    [네이버, 랩스, 디지털, 트윈, 기술, 구축, 실제, 서울, 상암, 지역, 정밀,...
504    [네이버, 클라우드, 네이버, 랩스, 자율, 주행, 시뮬레이션, 업체, 라이, 자율...
505    [모라, 이의, 자율, 주행, 시뮬레이션, 솔루션, 네이버, 랩스, 정밀, 지도, ...
506    [기차, 연착, 코레일, 확인, 기차, 이용, 승객, 여름, 휴가, 숙소, 배송, ...
507    [최수연, 대표, 대비, 비용, 절감, 계산기, 지도, 활용, 답변, 가능, 네이버...
Name: preprocessed_description, Length: 508, dtype: object

### BOW(Bag-of-Words) 생성

In [10]:
word_dict_news = corpora.Dictionary(NAVERMAP_news['preprocessed_description'])

In [11]:
word_dict_blog = corpora.Dictionary(NAVERMAP_blog['preprocessed_description'])

In [12]:
corpus_news = [word_dict_news.doc2bow(text) for text in NAVERMAP_news['preprocessed_description']]

In [13]:
corpus_blog = [word_dict_blog.doc2bow(text) for text in NAVERMAP_blog['preprocessed_description']]

In [14]:
corpus_news[2]

[(0, 1),
 (6, 5),
 (7, 4),
 (8, 1),
 (10, 1),
 (13, 1),
 (17, 2),
 (22, 6),
 (25, 9),
 (28, 1),
 (29, 2),
 (30, 2),
 (34, 9),
 (48, 1),
 (60, 1),
 (61, 3),
 (63, 4),
 (71, 1),
 (75, 1),
 (82, 1),
 (88, 2),
 (90, 4),
 (92, 1),
 (94, 2),
 (95, 1),
 (99, 1),
 (102, 2),
 (108, 1),
 (113, 1),
 (117, 3),
 (118, 1),
 (127, 1),
 (133, 14),
 (134, 1),
 (135, 1),
 (139, 1),
 (142, 1),
 (154, 6),
 (158, 2),
 (163, 1),
 (168, 4),
 (169, 2),
 (170, 1),
 (174, 2),
 (176, 1),
 (181, 1),
 (184, 3),
 (187, 7),
 (195, 1),
 (200, 3),
 (201, 3),
 (203, 1),
 (214, 1),
 (217, 1),
 (218, 1),
 (219, 2),
 (220, 2),
 (221, 1),
 (222, 1),
 (223, 1),
 (227, 1),
 (228, 1),
 (229, 1),
 (230, 1),
 (231, 1),
 (232, 2),
 (233, 1),
 (234, 1),
 (235, 1),
 (236, 1),
 (237, 1),
 (238, 1),
 (240, 3),
 (241, 1),
 (242, 2),
 (243, 1),
 (245, 1),
 (246, 2),
 (247, 1),
 (248, 1),
 (249, 1),
 (250, 1),
 (251, 1),
 (252, 4),
 (253, 1),
 (254, 1),
 (255, 1),
 (256, 1),
 (257, 1),
 (258, 1),
 (259, 1),
 (260, 1),
 (261, 1),
 (262,

In [15]:
[(idx,word) for idx, word in word_dict_news.items()]

[(0, '가까이'),
 (1, '가능성'),
 (2, '가량'),
 (3, '가운데'),
 (4, '간의'),
 (5, '갈수록'),
 (6, '검색'),
 (7, '격차'),
 (8, '결과'),
 (9, '결혼'),
 (10, '경우'),
 (11, '경제'),
 (12, '계속'),
 (13, '고객'),
 (14, '고도화'),
 (15, '공격'),
 (16, '과도'),
 (17, '관계자'),
 (18, '관련'),
 (19, '관리'),
 (20, '관심'),
 (21, '광고'),
 (22, '구글'),
 (23, '국가'),
 (24, '국기'),
 (25, '국내'),
 (26, '기간'),
 (27, '기관'),
 (28, '기록'),
 (29, '기반'),
 (30, '기준'),
 (31, '나스'),
 (32, '남북'),
 (33, '내면'),
 (34, '네이버'),
 (35, '노출'),
 (36, '녹색'),
 (37, '뉴스'),
 (38, '다양'),
 (39, '단가'),
 (40, '달리'),
 (41, '담당'),
 (42, '대가'),
 (43, '대부분'),
 (44, '대비'),
 (45, '대상'),
 (46, '대의'),
 (47, '대한'),
 (48, '도입'),
 (49, '디스플레이'),
 (50, '뜬금'),
 (51, '러시아'),
 (52, '로서'),
 (53, '로엔엔터테인먼트'),
 (54, '리다'),
 (55, '링크'),
 (56, '마찬가지'),
 (57, '마켓'),
 (58, '만원'),
 (59, '맥락'),
 (60, '메신저'),
 (61, '멜론'),
 (62, '모델'),
 (63, '모바일'),
 (64, '모습'),
 (65, '무료'),
 (66, '물음표'),
 (67, '뮤직'),
 (68, '미디어'),
 (69, '미래'),
 (70, '반년'),
 (71, '반면'),
 (72, '방문'),
 (73, '보이'),
 (74, '부분'),
 (75, '분석')

In [16]:
[(idx,word) for idx, word in word_dict_blog.items()]

[(0, '가능'),
 (1, '가지'),
 (2, '간석지'),
 (3, '거나'),
 (4, '건물'),
 (5, '검색'),
 (6, '결정'),
 (7, '경계'),
 (8, '경우'),
 (9, '계산'),
 (10, '계속'),
 (11, '곳도'),
 (12, '공간'),
 (13, '공부'),
 (14, '공적'),
 (15, '과수원'),
 (16, '관공서'),
 (17, '관련'),
 (18, '관리'),
 (19, '교부'),
 (20, '구분'),
 (21, '구비'),
 (22, '구축'),
 (23, '규칙'),
 (24, '그대로'),
 (25, '근무시간'),
 (26, '글자'),
 (27, '기록'),
 (28, '기본'),
 (29, '기재'),
 (30, '김해'),
 (31, '내야'),
 (32, '내용'),
 (33, '내집마련'),
 (34, '네이버'),
 (35, '누리'),
 (36, '다시'),
 (37, '단독'),
 (38, '단어'),
 (39, '대부분'),
 (40, '대표'),
 (41, '대하'),
 (42, '대한'),
 (43, '도공'),
 (44, '도로'),
 (45, '도면'),
 (46, '도움'),
 (47, '들판'),
 (48, '등록증'),
 (49, '등본'),
 (50, '라면'),
 (51, '로그인'),
 (52, '로딩'),
 (53, '링크'),
 (54, '마련'),
 (55, '마지막'),
 (56, '마찬가지'),
 (57, '먼저'),
 (58, '멘토'),
 (59, '면적'),
 (60, '명칭'),
 (61, '모든'),
 (62, '모바일'),
 (63, '목별'),
 (64, '목장'),
 (65, '목적'),
 (66, '목표'),
 (67, '무료'),
 (68, '무인'),
 (69, '문제'),
 (70, '민원'),
 (71, '발급'),
 (72, '방문'),
 (73, '방법'),
 (74, '버스'),
 (75, '버전'),
 (76, 

In [17]:
len(word_dict_news)

7052

### LDA 모델 훈련

#(1) 3개의 topic

#(2) LDA 모델을 생성. 'id2word'에는 단어 사전을 전달하고, 학습 반복 횟수는 15로 설정

#(3) 생성된 모델에서 각 주제에 대한 상위 4개 단어를 출력


In [27]:
N_TOPICS = 3 # (1) 
ldamodel_news = gensim.models.ldamodel.LdaModel(corpus_news, num_topics = N_TOPICS, id2word=word_dict_news, passes = 15) # (2)

topics = ldamodel_news.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.043*"네이버" + 0.017*"서비스" + 0.016*"검색" + 0.015*"지도"')
(1, '0.048*"네이버" + 0.020*"카카오" + 0.017*"서비스" + 0.014*"지도"')
(2, '0.029*"네이버" + 0.016*"정보" + 0.014*"지도" + 0.012*"서비스"')


In [28]:
N_TOPICS = 3 # (1) 
ldamodel_blog = gensim.models.ldamodel.LdaModel(corpus_blog, num_topics = N_TOPICS, id2word=word_dict_blog, passes = 15) # (2)

topics = ldamodel_blog.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.051*"지도" + 0.036*"네이버" + 0.020*"확인" + 0.017*"거리"')
(1, '0.039*"네이버" + 0.025*"지도" + 0.021*"등록" + 0.015*"레이스"')
(2, '0.042*"지도" + 0.039*"네이버" + 0.014*"검색" + 0.014*"장소"')


### LDA 시각화

In [29]:
vis_news = pyLDAvis.gensim_models.prepare(ldamodel_news, corpus_news, word_dict_news)
pyLDAvis.display(vis_news)

In [30]:
vis_blog = pyLDAvis.gensim_models.prepare(ldamodel_blog, corpus_blog, word_dict_blog)
pyLDAvis.display(vis_blog)